In [ ]:
-- #1 
SELECT TOP (10) WITH TIES
    A.Name AS [Artist Name]
    ,COUNT(A.Name) * T.UnitPrice AS [Total Sales]
FROM Artist A
JOIN Album Al
    ON Al.ArtistId = A.ArtistId
JOIN Track T
    ON T.AlbumId = Al.AlbumId
JOIN InvoiceLine IL
    ON IL.TrackId = T.TrackId
JOIN Invoice I
    ON I.InvoiceId = IL.InvoiceId
WHERE I.InvoiceDate BETWEEN '7/1/2011' AND '6/30/2012'
    AND T.MediaTypeId != 3
GROUP BY A.Name, T.UnitPrice
ORDER BY [Total Sales] DESC